In [2]:
%cd ..

c:\Users\ihhim\OneDrive\Desktop\project1


In [3]:
import pandas as pd
from PIL import Image
import torch
import os

In [39]:
data_map=pd.read_csv(r'C:\Users\ihhim\OneDrive\Desktop\project1\data\chest_xray\metadata.csv')

In [41]:
len(data_map)

8390

In [6]:
from sklearn.preprocessing import LabelEncoder

In [42]:
le = LabelEncoder()
data_map['encoded_class'] = le.fit_transform(data_map['class'])

In [45]:
data_map

,image_path,class,split,encoded_class
0,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,NORMAL,train,0
1,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,train,1
2,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,train,1
3,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,NORMAL,train,0
4,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,NORMAL,train,0
...,...,...,...,...
8385,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,test,1
8386,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,test,1
8387,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,test,1
8388,C:\Users\ihhim\OneDrive\Desktop\project1\data\...,PNEUMONIA,test,1


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms,models

In [43]:
class xraydataset(Dataset):
    def __init__(self,df_split,transform):
        self.df_split=df_split
        self.transform=transform
        
    def __len__(self):
        return len(self.df_split)  
    
    def __getitem__(self,index):  
        img_path=self.df_split.iloc[index]['image_path']
        label=self.df_split.iloc[index]['encoded_class']

        image=Image.open(img_path).convert('L')
        img_trans=self.transform(image)

        return img_trans,label

        

In [46]:
train_dataset=xraydataset(data_map[data_map['split']=='train'],transform=transforms.ToTensor())
val_dataset=xraydataset(data_map[data_map['split']=='val'],transform=transforms.ToTensor())

In [47]:
train_dataset[0]

(tensor([[[0.0039, 0.0275, 0.0863,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0196, 0.0784,  ..., 0.0118, 0.0118, 0.0118],
          [0.0000, 0.0118, 0.0667,  ..., 0.9765, 0.9569, 0.9412],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]]),
 0)

In [48]:
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=32,shuffle=False)

In [49]:
model=models.resnet18(models.ResNet18_Weights.IMAGENET1K_V1)


c:\Users\ihhim\anaconda3\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [50]:
model.conv1=nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
model.fc=nn.Linear(in_features=512,out_features=2,bias=True)

In [51]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [52]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [53]:
device

device(type='cuda')

In [54]:
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [57]:
class_counts = data_map[data_map['split']=='train']['encoded_class'].value_counts().sort_index().values
weights = torch.tensor(1.0 / class_counts, dtype=torch.float).to(device)
loss = nn.CrossEntropyLoss(weight=weights)
optimizer=optim.Adam(model.parameters(),lr=0.0001)

In [60]:
from tqdm import tqdm
num_epochs=10
for epoch in range(num_epochs):
    model.train()
    running_loss=0
    for images,labels in tqdm(train_dataloader,total=len(train_dataloader)):
        images,labels=images.to(device),labels.to(device).long()
        optimizer.zero_grad()
        preds=model(images)
        loss_val=loss(preds,labels)
        loss_val.backward()
        optimizer.step()
        running_loss+=loss_val.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader):.4f}")

  

100%|██████████| 243/243 [00:19<00:00, 12.47it/s]


Epoch 1/10, Loss: 0.0097


100%|██████████| 243/243 [00:17<00:00, 14.15it/s]


Epoch 2/10, Loss: 0.0044


100%|██████████| 243/243 [00:18<00:00, 13.06it/s]


...

Epoch 8/10, Loss: 0.0000


100%|██████████| 243/243 [00:18<00:00, 13.25it/s]


Epoch 9/10, Loss: 0.0000


100%|██████████| 243/243 [00:18<00:00, 13.39it/s]

Epoch 10/10, Loss: 0.0000


In [61]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=le.classes_))

              precision    recall  f1-score   support

      NORMAL       1.00      0.88      0.93         8
   PNEUMONIA       0.89      1.00      0.94         8

    accuracy                           0.94        16
   macro avg       0.94      0.94      0.94        16
weighted avg       0.94      0.94      0.94        16



In [62]:
os.makedirs(r'C:\Users\ihhim\OneDrive\Desktop\project1\models', exist_ok=True)
torch.save(model.state_dict(), r'C:\Users\ihhim\OneDrive\Desktop\project1\models\resnet18.pth')

In [ ]:
def infer(img):
    img_gray=img.convert('L')
    img_trans=transforms.ToTensor()(img).unsqueeze(0).to(device)
    preds=model(img_trans)
    print(preds)
    class_idx=torch.argmax(preds,dim=1).item()
    print(class_idx)
    class_name=le.inverse_transform([class_idx])[0]
    print(class_name) 

img_path=r'C:\Users\ihhim\OneDrive\Desktop\project1\data\chest_xray\test\NORMAL\NORMAL2-IM-0374-0001.jpeg'
img=Image.open(img_path)

tensor([[-0.0944, -0.2634]], device='cuda:0', grad_fn=<AddmmBackward0>)
0
NORMAL
